In [1]:
import sys

import pandas as pd
from pathlib import Path

In [2]:
project_parent = Path('./').absolute().parent
src_pth = project_parent/'src'

src_pth.exists()

True

In [3]:
# import the project package from the project package path
# ideally will be imported using 'from arcgis import da'
sys.path.insert(0, str(src_pth))

In [4]:
import dm

# load the "autoreload" extension so as you change code in src, it gets reloaded
%load_ext autoreload
%autoreload 2

D:\projects\demographic-modeling-module\src\dm\_modify_geoaccessor.py:31: UserWarning: registration of accessor <class 'dm._modify_geoaccessor.GeoAccessorIO'> under name 'spatial' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexistingattribute with the same name.
  class GeoAccessorIO(GeoAccessor):


In [5]:
usa = dm.Country('USA')

In [6]:
enrich_vars = usa.enrich_variables[(usa.enrich_variables.data_collection == 'KeyUSFacts') & (usa.enrich_variables.vintage == '2019')].enrich_str

enrich_vars

6450     KeyUSFacts.TOTPOP_CY
6452      KeyUSFacts.GQPOP_CY
6453    KeyUSFacts.DIVINDX_CY
6456      KeyUSFacts.TOTHH_CY
6458    KeyUSFacts.AVGHHSZ_CY
6459    KeyUSFacts.MEDHINC_CY
6461    KeyUSFacts.AVGHINC_CY
6463        KeyUSFacts.PCI_CY
6467      KeyUSFacts.TOTHU_CY
6469      KeyUSFacts.OWNER_CY
6471     KeyUSFacts.RENTER_CY
6473     KeyUSFacts.VACANT_CY
6475     KeyUSFacts.MEDVAL_CY
6477     KeyUSFacts.AVGVAL_CY
6479    KeyUSFacts.POPGRW10CY
6480     KeyUSFacts.HHGRW10CY
6481    KeyUSFacts.FAMGRW10CY
6487       KeyUSFacts.DPOP_CY
6488    KeyUSFacts.DPOPWRK_CY
6489    KeyUSFacts.DPOPRES_CY
Name: enrich_str, dtype: object

In [8]:
%%time
bg_enrich_df = usa.cbsas.get('seattle').level(0).get().spatial.enrich(enrich_vars)

bg_enrich_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2474 entries, 0 to 2473
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   ID                     2474 non-null   object  
 1   NAME                   2474 non-null   object  
 2   KeyUSFacts_TOTPOP_CY   2474 non-null   float64 
 3   KeyUSFacts_GQPOP_CY    2474 non-null   float64 
 4   KeyUSFacts_DIVINDX_CY  2474 non-null   float64 
 5   KeyUSFacts_TOTHH_CY    2474 non-null   float64 
 6   KeyUSFacts_AVGHHSZ_CY  2474 non-null   float64 
 7   KeyUSFacts_MEDHINC_CY  2474 non-null   float64 
 8   KeyUSFacts_AVGHINC_CY  2474 non-null   float64 
 9   KeyUSFacts_PCI_CY      2474 non-null   float64 
 10  KeyUSFacts_TOTHU_CY    2474 non-null   float64 
 11  KeyUSFacts_OWNER_CY    2474 non-null   float64 
 12  KeyUSFacts_RENTER_CY   2474 non-null   float64 
 13  KeyUSFacts_VACANT_CY   2474 non-null   float64 
 14  KeyUSFacts_MEDVAL_CY   2474 non-null   f